In [3]:
import pandas as pd

In [4]:
train_data = pd.read_csv("/opt/ml/input/data/train/train_ratings.csv")
titles = pd.read_csv("/opt/ml/input/data/train/titles.tsv", sep="\t")

In [5]:
titles

,item,title
0,318,"Shawshank Redemption, The (1994)"
1,2571,"Matrix, The (1999)"
2,2959,Fight Club (1999)
3,296,Pulp Fiction (1994)
4,356,Forrest Gump (1994)
...,...,...
6802,73106,American Pie Presents: The Book of Love (Ameri...
6803,109850,Need for Speed (2014)
6804,8605,Taxi 3 (2003)
6805,3689,Porky's II: The Next Day (1983)


# 가설 : 모든 title은 고유하다

In [6]:
titles.title = titles.title.str.lower()

In [7]:
titles.title.nunique()

6806

In [8]:
tmp = titles.groupby("title").size()
tmp.sort_values()   # 하나의 중복된 title이 있다. - War of the Worlds (2005)

title
"great performances" cats (1998)             1
overboard (1987)                             1
over the hedge (2006)                        1
outsourced (2006)                            1
outsiders, the (1983)                        1
                                            ..
freaky friday (1977)                         1
freaks (1932)                                1
freeway (1996)                               1
à nous la liberté (freedom for us) (1931)    1
war of the worlds (2005)                     2
Length: 6806, dtype: int64

In [9]:
titles[titles.title == "war of the worlds (2005)"]  # 같은 title, 다른 item 번호

,item,title
349,34048,war of the worlds (2005)
1926,64997,war of the worlds (2005)


In [10]:
directors = pd.read_csv("/opt/ml/input/data/train/directors.tsv", sep="\t")
genres = pd.read_csv("/opt/ml/input/data/train/genres.tsv", sep="\t")
writers = pd.read_csv("/opt/ml/input/data/train/writers.tsv", sep="\t")
years = pd.read_csv("/opt/ml/input/data/train/years.tsv", sep="\t")

In [11]:
print(directors[directors.item == 64997])
print(directors[directors.item == 34048])
print()
print(genres[genres.item == 64997])
print(genres[genres.item == 34048])
print()
print(writers[writers.item == 64997])
print(writers[writers.item == 34048])
print()
print(years[years.item == 64997])
print(years[years.item == 34048])
# item 번호 34048이 64997을 포함하는 모습을 보인다.

Empty DataFrame
Columns: [item, director]
Index: []
      item   director
384  34048  nm0000229

       item   genre
4735  64997  Action
4736  64997  Sci-Fi
       item      genre
1001  34048     Action
1002  34048  Adventure
1003  34048     Sci-Fi
1004  34048   Thriller

        item     writer
10158  64997  nm0920229
        item     writer
4527   34048  nm0295264
6221   34048  nm0462895
10154  34048  nm0920229

       item  year
4806  64997  2005
       item  year
4751  34048  2005


In [12]:
titles[titles.title == "war of the worlds (2005)"]

,item,title
349,34048,war of the worlds (2005)
1926,64997,war of the worlds (2005)


In [13]:
train_title = train_data.merge(titles, on="item", how="left")

In [14]:
train_title

,user,item,time,title
0,11,4643,1230782529,planet of the apes (2001)
1,11,170,1230782534,hackers (1995)
2,11,531,1230782539,"secret garden, the (1993)"
3,11,616,1230782542,"aristocats, the (1970)"
4,11,2140,1230782563,"dark crystal, the (1982)"
...,...,...,...,...
5154466,138493,44022,1260209449,ice age 2: the meltdown (2006)
5154467,138493,4958,1260209482,behind enemy lines (2001)
5154468,138493,68319,1260209720,x-men origins: wolverine (2009)
5154469,138493,40819,1260209726,walk the line (2005)


In [15]:
train_title[train_title.title == "war of the worlds (2005)"].groupby("item").size()

item
34048    3022
64997     178
dtype: int64

# 가설 : 모든 영화 제목에는 올바른 year 정보가 포함되어있다.

In [16]:
new_titles = titles.copy()

In [17]:
# 제목에서 가장 마지막에 있는 year 추출하여 "_year" column에 추가
new_titles["_year"] = new_titles.title.apply(lambda x: x.split()[-1])
new_titles["_year"] = new_titles._year.apply(lambda x: x[1:5])

In [18]:
new_titles

,item,title,_year
0,318,"shawshank redemption, the (1994)",1994
1,2571,"matrix, the (1999)",1999
2,2959,fight club (1999),1999
3,296,pulp fiction (1994),1994
4,356,forrest gump (1994),1994
...,...,...,...
6802,73106,american pie presents: the book of love (ameri...,2009
6803,109850,need for speed (2014),2014
6804,8605,taxi 3 (2003),2003
6805,3689,porky's ii: the next day (1983),1983


In [19]:
years = pd.read_csv("/opt/ml/input/data/train/years.tsv", sep="\t")

In [20]:
new_titles = new_titles.merge(years, on="item", how="left")

In [21]:
# year.tsv의 year 정보와 title에서 추출한 year 정보가 일치하는지 확인하기 위해 형변환
new_titles._year = new_titles._year.apply(pd.to_numeric)
new_titles._year = new_titles._year.astype(float)

In [22]:
new_titles.dtypes

item       int64
title     object
_year    float64
year     float64
dtype: object

In [23]:
new_titles

,item,title,_year,year
0,318,"shawshank redemption, the (1994)",1994.0,1994.0
1,2571,"matrix, the (1999)",1999.0,1999.0
2,2959,fight club (1999),1999.0,1999.0
3,296,pulp fiction (1994),1994.0,1994.0
4,356,forrest gump (1994),1994.0,1994.0
...,...,...,...,...
6802,73106,american pie presents: the book of love (ameri...,2009.0,2009.0
6803,109850,need for speed (2014),2014.0,2014.0
6804,8605,taxi 3 (2003),2003.0,2003.0
6805,3689,porky's ii: the next day (1983),1983.0,1983.0


In [24]:
new_titles[new_titles._year == new_titles.year]

,item,title,_year,year
0,318,"shawshank redemption, the (1994)",1994.0,1994.0
1,2571,"matrix, the (1999)",1999.0,1999.0
2,2959,fight club (1999),1999.0,1999.0
3,296,pulp fiction (1994),1994.0,1994.0
4,356,forrest gump (1994),1994.0,1994.0
...,...,...,...,...
6802,73106,american pie presents: the book of love (ameri...,2009.0,2009.0
6803,109850,need for speed (2014),2014.0,2014.0
6804,8605,taxi 3 (2003),2003.0,2003.0
6805,3689,porky's ii: the next day (1983),1983.0,1983.0


In [25]:
# year이 다른 경우는 NaN인 경우밖에 없다. -> year 결측값 채울 수 있다.
new_titles[new_titles._year != new_titles.year]

,item,title,_year,year
984,6987,"cabinet of dr. caligari, the (cabinet des dr. ...",1920.0,NaN
1284,3310,"kid, the (1921)",1921.0,NaN
2359,7065,"birth of a nation, the (1915)",1915.0,NaN
2620,7243,intolerance: love's struggle throughout the ag...,1916.0,NaN
2669,8511,"immigrant, the (1917)",1917.0,NaN
2676,6988,broken blossoms or the yellow man and the girl...,1919.0,NaN
3177,32898,"trip to the moon, a (voyage dans la lune, le) ...",1902.0,NaN
6043,119145,kingsman: the secret service (2015),2015.0,NaN


In [26]:
new_titles.drop("year", axis=1, inplace=True)

In [27]:
new_titles.rename(columns={"_year": "year"}, inplace=True)
new_titles.year = new_titles.year.astype(int)

In [28]:
new_titles

,item,title,year
0,318,"shawshank redemption, the (1994)",1994
1,2571,"matrix, the (1999)",1999
2,2959,fight club (1999),1999
3,296,pulp fiction (1994),1994
4,356,forrest gump (1994),1994
...,...,...,...
6802,73106,american pie presents: the book of love (ameri...,2009
6803,109850,need for speed (2014),2014
6804,8605,taxi 3 (2003),2003
6805,3689,porky's ii: the next day (1983),1983


# title 유사도
title의 유사도 비교를 위해 TF-IDF를 사용하였다.

In [29]:
# !pip install scikit-learn

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [31]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(new_titles.title)

In [32]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [33]:
for idx in range(len(new_titles)):
    for i in range(len(new_titles)):
        if new_titles.iloc[i].title != new_titles.iloc[idx].title\
            and 0.9 < cosine_sim[idx][i] < 1.0:
            print("==== SIMILAR! ====")
            print(new_titles.iloc[idx].title, new_titles.iloc[idx].year)
            print(new_titles.iloc[i].title, new_titles.iloc[i].year) 
            print(cosine_sim[idx][i])
            print("===================")

In [34]:
new_titles.title = new_titles.title.astype("string")

In [35]:
new_titles.dtypes

item      int64
title    string
year      int64
dtype: object

순서만 바뀐 제목 확인을 위해 countvectorizer을 사용하였다.

In [36]:
from sklearn.feature_extraction.text import CountVectorizer

In [37]:
word_list = []

vect = CountVectorizer()

for i in range(len(new_titles)):
    vect.fit_transform([new_titles.iloc[i].title])
    word_list.append(vect.vocabulary_)

In [38]:
# 앞서 두 개의 title이 중복된 경우를 제외하고는 순서만 바뀐 경우는 없었다.
for i in range(len(word_list) - 1):
    for j in range(i+1, len(word_list)):
        if word_list[i] == word_list[j]:
            print(word_list[i], word_list[j])

{'war': 3, 'of': 1, 'the': 2, 'worlds': 4, '2005': 0} {'war': 3, 'of': 1, 'the': 2, 'worlds': 4, '2005': 0}
{'2004': 0} {'2004': 0}
{'2001': 0} {'2001': 0}


sentence transformers

In [40]:
# !pip install sentence_transformers

In [41]:
from sentence_transformers import SentenceTransformer, util

In [42]:
movie_titles = titles.title.values
model = SentenceTransformer('paraphrase-distilroberta-base-v1')
embeddings = model.encode(movie_titles, convert_to_tensor=True)

- paraphrase-multilingual-MiniLM-L12-v2 -> paraphrase-distilroberta-base-v1보다 전체적으로 유사도를 높게 측정하는 것으로 보임.

In [43]:
movie_titles = titles.title.values
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(movie_titles, convert_to_tensor=True)

In [44]:
cosine_scores = util.pytorch_cos_sim(embeddings, embeddings)

In [ ]:
for idx in range(len(cosine_scores)): ###110
    for cosval, name in zip(cosine_scores[idx],movie_titles):
        if movie_titles[idx] != name and 0.80 < cosval < 1.0:
            print(f"{cosval*100}%\t\t{movie_titles[idx]}----{name}")

- paraphrase-MiniLM-L6-v2

In [ ]:
movie_titles = titles.title.values
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
embeddings = model.encode(movie_titles, convert_to_tensor=True)

In [ ]:
cosine_scores = util.pytorch_cos_sim(embeddings, embeddings)
for idx in range(len(cosine_scores)): ###110
    for cosval, name in zip(cosine_scores[idx],movie_titles):
        if movie_titles[idx] != name and 0.80 < cosval < 1.0:
            print(f"{cosval*100}%\t\t{movie_titles[idx]}----{name}")

- paraphrase-distilroberta-base-v1

In [ ]:
movie_titles = titles.title.values
model = SentenceTransformer('paraphrase-distilroberta-base-v1')
embeddings = model.encode(movie_titles, convert_to_tensor=True)

In [ ]:
cosine_scores = util.pytorch_cos_sim(embeddings, embeddings)
cosine_scores

tensor([[ 1.0000,  0.2723,  0.2219,  ...,  0.1206,  0.2550,  0.1470],
        [ 0.2723,  1.0000,  0.4619,  ...,  0.1824,  0.2633,  0.4373],
        [ 0.2219,  0.4619,  1.0000,  ...,  0.0987,  0.2437,  0.3924],
        ...,
        [ 0.1206,  0.1824,  0.0987,  ...,  1.0000,  0.2284, -0.0154],
        [ 0.2550,  0.2633,  0.2437,  ...,  0.2284,  1.0000,  0.2208],
        [ 0.1470,  0.4373,  0.3924,  ..., -0.0154,  0.2208,  1.0000]],
       device='cuda:0')

In [ ]:
for idx in range(len(cosine_scores)): ###110
    for cosval, name in zip(cosine_scores[idx],movie_titles):
        if 0.80 < cosval < 1.0:
            print(f"{cosval*100}%\t\t{movie_titles[idx]}----{name}")